<a href="https://colab.research.google.com/github/linbojun/CSCI1430_Colorization/blob/main/chromaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np


In [4]:
class discriminator(tf.keras.Model):
  def __init__(self):
    super(discriminator, self).__init__()
    self.conv1 = tf.nn.Conv2d(3, 64, filters=(4,4),stride=(2,2), padding=1) # 64, 112, 112
    self.conv2 = tf.nn.Conv2d(64, 128, filters=(4,4),stride=(2,2), padding=1) # 128, 56, 56
    self.conv3 = tf.nn.Conv2d(128,256, filters=(4,4),stride=(2,2), padding=1) # 256, 28, 28, 2
    self.conv4 = tf.nn.Conv2d(256,512, filters=(4,4),stride=(2,2), padding=1)# 512, 28, 28
    self.conv5 = tf.nn.Conv2d(512,1, filters=(4,4),stride=(2,2), padding=1) # 1, 
    self.relu = tf.nn.LeakyReLU(0.3)

  def call(self, inputs):
    token = self.conv1(inputs)               #[-1, 64, 112, 112]
    token = self.relu(token)                 #[-1, 64, 112, 112]    
    token = self.conv2(token)                #[-1, 128, 56, 56] 
    token = self.relu(token)                 #[-1, 128, 56, 56] 
    token = self.conv3(token)                #[-1, 256, 28, 28]
    token = self.relu(token)                 #[-1, 256, 28, 28]   
    token = self.conv4(token)                #[-1, 512, 27, 27]
    token = self.relu(token)                 #[-1, 512, 27, 27]
    token = self.conv5(token)                #[-1, 1, 26, 26]
    return net



In [ ]:
class generator(tf.keras.Model):
  def __init__(self):
    super(generator, self).__init__()
    self.VGG = tf.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    self.VGG = tf.keras.models.Model(VGG_model.input,VGG_model.layers[-6].output)
    # Global Features

    self.global_conv1 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', strides=(2, 2), activation='relu')
    self.global_conv2 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', strides=(1, 1), activation='relu')

    self.global_conv3 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', strides=(2, 2), activation='relu')
    self.global_conv4 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', strides=(1, 1), activation='relu')

    self.flatten = tf.keras.layers.Flatten()
    self.global_dense1 = tf.keras.layers.Dense(1024)
    self.global_dense2 = tf.keras.layers.Dense(512)
    self.global_dense3 = tf.keras.layers.Dense(256)
    self.global_repeat = tf.keras.layers.RepeatVector(28*28)
    self.global_reshape = tf.keras.layers.Reshape((28,28, 256))

    # self.global_dense4 = tf.keras.layers.Dense(4096)
    # self.global_dense5 = tf.keras.layers.Dense(4096)
    # self.global_dense6 = tf.keras.layers.Dense(1000, activation='softmax')
    self.batchnorm = tf.keras.layers.BatchNormalization()

    # Midlevel Features

    self.mid_conv1 = tf.keras.layers.Conv2D(512, (3, 3),  padding='same', strides=(1, 1), activation='relu')
    self.mid_conv2 = tf.keras.layers.Conv2D(256, (3, 3),  padding='same', strides=(1, 1), activation='relu')

    # fusion of (VGG16 + Midlevel) + (VGG16 + Global)
    self.fusion = tf.keras.layers.concatenate
    
    # Fusion + Colorization
    self.out_conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', strides=(1, 1), activation='relu')
    self.out_conv2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', strides=(1, 1), activation='relu')
    self.out_up_sample = keras.layers.UpSampling2D(size=(2,2))
    self.out_conv3 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', strides=(1, 1), activation='relu')
    self.out_conv4 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', strides=(1, 1), activation='relu')
    self.out_conv5 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', strides=(1, 1), activation='relu')
    self.out_conv6 = tf.keras.layers.Conv2D(2, (3, 3), padding='same', strides=(1, 1), activation='sigmoid')

  def call(self, inputs):
    vgg_res = self.VGG(inputs)

    #global feature
    global_feat= self.global_conv1(vgg_res)
    global_feat = self.batchnorm(global_feat)

    global_feat = self.global_conv2(global_feat)
    global_feat = self.batchnorm(global_feat)

    global_feat = self.global_conv3(global_feat)
    global_feat = self.batchnorm(global_feat)

    global_feat = self.global_conv4(global_feat)
    global_feat = self.batchnorm(global_feat)

    global_feat2 = self.flatten(global_feat)

    global_feat2 = self.global_dense1(global_feat2)
    global_feat2 = self.global_dense2(global_feat2)
    global_feat2 = self.global_dense3(global_feat2)
    global_feat2 = self.global_repeat(global_feat2)

    #mid feature
    mid_feat = self.mid_conv1(vgg_res)
    mid_feat = self.batchnorm(mid_feat)
    mid_feat = self.mid_conv2(vgg_res)
    mid_feat = self.batchnorm(mid_feat)

    #fusion
    fusion_feat =  self.fusion([mid_feat, global_feat2])

    #output
    out = self.out_conv1(fusion_feat)
    out = self.out_conv2(out)

    out = self.out_up_sample(out)
    out = self.out_conv3(out)
    out = self.out_conv4(out)

    out = self.out_up_sample(out)
    out = self.out_conv5(out)
    out = self.out_conv6(out)

    out = self.out_up_sample(out)
    return out;


    
    